In [1]:
#Library Importing

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

# INITIALIZING SPARKSESSION

In [2]:
#Initialization of a Spark Session
sc = SparkSession.builder.appName("PySparkExample")\
    .config ("spark.sql.shuffle.partitions", "50") \
    .config("spark.driver.maxResultSize","5g") \
    .config ("spark.sql.execution.arrow.enabled", "true")\
    .getOrCreate()

# CREATING DATAFRAMES

In [3]:
#Read from json, parquet, txt, csv formatted files.
#Creates a spark data frame called as raw_data.

#JSON
dataframe = sc.read.json('dataset/nyt2.json')

#TXT FILES
# dataframe_txt = sc.read.text('text_data.txt')

#CSV FILES
# dataframe_csv = sc.read.csv('csv_data.csv')

#PARQUET FILES
# dataframe_parquet = sc.read.load('parquet_data.parquet')


In [4]:
dataframe = sc.read.json('dataset/nyt2.json')

In [5]:
dataframe.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|            THE HOST|          [3]|


# DUPLICATE VALUES

In [6]:
dataframe_dropdup = dataframe.dropDuplicates()

In [7]:
dataframe_dropdup.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+----------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|   price|   published_date|       publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+----------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|Clive Cussler wit...|[[1213401600000]]|Juan Cabrillo and...|[26.95,]|[[1214697600000]]|          Putnam| [4]|           [3]|         PLAGUE SHIP|          [2]|
|[5b4aa4ead3089013...|http://www.amazon...|      Jeffery Deaver|[[1215820800000]]|Detectives Lincol...|   [, 0]|[[1217116800000]]|Simon & Schuster|[20]|           [0]|   THE BROKEN WINDOW|

In [8]:
dataframe.columns

['_id',
 'amazon_product_url',
 'author',
 'bestsellers_date',
 'description',
 'price',
 'published_date',
 'publisher',
 'rank',
 'rank_last_week',
 'title',
 'weeks_on_list']

# QUERIES

# Select

In [9]:
# Select Operation

dataframe.select("author").show(10) #Show all entries in title column

dataframe.select("author", "title", "rank", "price").show(10)  #Show all entries in title, author, rank, price columns

+--------------------+
|              author|
+--------------------+
|       Dean R Koontz|
|     Stephenie Meyer|
|        Emily Giffin|
|   Patricia Cornwell|
|     Chuck Palahniuk|
|James Patterson a...|
|       John Sandford|
|       Jimmy Buffett|
|    Elizabeth George|
|      David Baldacci|
+--------------------+
only showing top 10 rows

+--------------------+--------------------+----+--------+
|              author|               title|rank|   price|
+--------------------+--------------------+----+--------+
|       Dean R Koontz|           ODD HOURS| [1]|  [, 27]|
|     Stephenie Meyer|            THE HOST| [2]|[25.99,]|
|        Emily Giffin|LOVE THE ONE YOU'...| [3]|[24.95,]|
|   Patricia Cornwell|           THE FRONT| [4]|[22.95,]|
|     Chuck Palahniuk|               SNUFF| [5]|[24.95,]|
|James Patterson a...|SUNDAYS AT TIFFANY’S| [6]|[24.99,]|
|       John Sandford|        PHANTOM PREY| [7]|[26.95,]|
|       Jimmy Buffett|          SWINE NOT?| [8]|[21.99,]|
|    Elizabeth

# When

In [10]:
# Show title and assign 0 or 1 depending on title

dataframe.select("title", when(dataframe.title != 'ODD HOURS', 1).otherwise(0)).show(10)

+--------------------+-----------------------------------------------------+
|               title|CASE WHEN (NOT (title = ODD HOURS)) THEN 1 ELSE 0 END|
+--------------------+-----------------------------------------------------+
|           ODD HOURS|                                                    0|
|            THE HOST|                                                    1|
|LOVE THE ONE YOU'...|                                                    1|
|           THE FRONT|                                                    1|
|               SNUFF|                                                    1|
|SUNDAYS AT TIFFANY’S|                                                    1|
|        PHANTOM PREY|                                                    1|
|          SWINE NOT?|                                                    1|
|     CARELESS IN RED|                                                    1|
|     THE WHOLE TRUTH|                                                    1|

In [11]:
# Show rows with specified authors if in the given options

dataframe [dataframe.author.isin("John Sandford","Emily Giffin")].show(5)


+--------------------+--------------------+-------------+-----------------+--------------------+--------+-----------------+------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|       author| bestsellers_date|         description|   price|   published_date|   publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-------------+-----------------+--------------------+--------+-----------------+------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...| Emily Giffin|[[1211587200000]]|A woman's happy m...|[24.95,]|[[1212883200000]]|St. Martin's| [3]|           [2]|LOVE THE ONE YOU'...|          [2]|
|[5b4aa4ead3089013...|http://www.amazon...|John Sandford|[[1211587200000]]|The Minneapolis d...|[26.95,]|[[1212883200000]]|      Putnam| [7]|           [4]|        PHANTOM PREY|          [3]|
|[5b4aa4ead3089013...|http://www.amazon.

# Like

In [12]:
# Show author and title is TRUE if title has " THE " word in titles

dataframe.select("author", "title", dataframe.title.like("% THE %")).show(15)

+--------------------+--------------------+------------------+
|              author|               title|title LIKE % THE %|
+--------------------+--------------------+------------------+
|       Dean R Koontz|           ODD HOURS|             false|
|     Stephenie Meyer|            THE HOST|             false|
|        Emily Giffin|LOVE THE ONE YOU'...|              true|
|   Patricia Cornwell|           THE FRONT|             false|
|     Chuck Palahniuk|               SNUFF|             false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|             false|
|       John Sandford|        PHANTOM PREY|             false|
|       Jimmy Buffett|          SWINE NOT?|             false|
|    Elizabeth George|     CARELESS IN RED|             false|
|      David Baldacci|     THE WHOLE TRUTH|             false|
|        Troy Denning|          INVINCIBLE|             false|
|          James Frey|BRIGHT SHINY MORNING|             false|
|         Garth Stein|THE ART OF RACING...|            

# StartsWith - EndsWith

In [13]:
dataframe.select("author", "title", dataframe.title.startswith("THE")).show(5)

dataframe.select("author", "title", dataframe.title.endswith("NT")).show(5)


+-----------------+--------------------+----------------------+
|           author|               title|startswith(title, THE)|
+-----------------+--------------------+----------------------+
|    Dean R Koontz|           ODD HOURS|                 false|
|  Stephenie Meyer|            THE HOST|                  true|
|     Emily Giffin|LOVE THE ONE YOU'...|                 false|
|Patricia Cornwell|           THE FRONT|                  true|
|  Chuck Palahniuk|               SNUFF|                 false|
+-----------------+--------------------+----------------------+
only showing top 5 rows

+-----------------+--------------------+-------------------+
|           author|               title|endswith(title, NT)|
+-----------------+--------------------+-------------------+
|    Dean R Koontz|           ODD HOURS|              false|
|  Stephenie Meyer|            THE HOST|              false|
|     Emily Giffin|LOVE THE ONE YOU'...|              false|
|Patricia Cornwell|           THE

# Substring

In [14]:
dataframe.select(dataframe.author.substr(1, 6).alias("title")).show()

+------+
| title|
+------+
|Dean R|
|Stephe|
|Emily |
|Patric|
|Chuck |
|James |
|John S|
|Jimmy |
|Elizab|
|David |
|Troy D|
|James |
|Garth |
|Debbie|
|Jeff S|
|Philli|
|Jhumpa|
|Joseph|
|John G|
|James |
+------+
only showing top 20 rows



In [15]:
dataframe.select(dataframe.author.substr(1, 3).alias("title")).show(5)

dataframe.select(dataframe.author.substr(3, 6).alias("title")).show(5)

dataframe.select(dataframe.author.substr(1, 6).alias("title")).show(5)

+-----+
|title|
+-----+
|  Dea|
|  Ste|
|  Emi|
|  Pat|
|  Chu|
+-----+
only showing top 5 rows

+------+
| title|
+------+
|an R K|
|epheni|
|ily Gi|
|tricia|
|uck Pa|
+------+
only showing top 5 rows

+------+
| title|
+------+
|Dean R|
|Stephe|
|Emily |
|Patric|
|Chuck |
+------+
only showing top 5 rows



# INSPECT DATA

In [16]:
# Returns dataframe column names and data types
dataframe.dtypes

[('_id', 'struct<$oid:string>'),
 ('amazon_product_url', 'string'),
 ('author', 'string'),
 ('bestsellers_date', 'struct<$date:struct<$numberLong:string>>'),
 ('description', 'string'),
 ('price', 'struct<$numberDouble:string,$numberInt:string>'),
 ('published_date', 'struct<$date:struct<$numberLong:string>>'),
 ('publisher', 'string'),
 ('rank', 'struct<$numberInt:string>'),
 ('rank_last_week', 'struct<$numberInt:string>'),
 ('title', 'string'),
 ('weeks_on_list', 'struct<$numberInt:string>')]

In [195]:
# Displays the content of dataframe
dataframe.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+--------------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|   price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+--------------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|              Bantam| [1]|           [0]|           ODD HOURS|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|       Little, Brown| [2]|           [1]| 

In [17]:
# Return first n rows
dataframe.head()

Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'))

In [18]:
# Returns first row
dataframe.first()

Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'))

In [19]:
# Return first n rows
dataframe.take(5)

[Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1')),
 Row(_id=Row($oid='5b4aa4ead3089013507db18c'), amazon_product_url='http://www.amazon.com/The-Host-Novel-Stephenie-Meyer/dp/0316218502?tag=NYTBS-20', author='Stephenie Meyer', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Aliens have taken control of the minds and bodies of most humans, but one woman won’t surrender.', price=Row($numberDouble='25.99', $numberInt=None), published_date=

In [20]:
# Computes summary statistics
dataframe.describe().show()

+-------+--------------------+---------------+--------------------+---------+------------------+
|summary|  amazon_product_url|         author|         description|publisher|             title|
+-------+--------------------+---------------+--------------------+---------+------------------+
|  count|               10195|          10195|               10195|    10195|             10195|
|   mean|                null|           null|                null|     null|1877.7142857142858|
| stddev|                null|           null|                null|     null| 370.9760613506458|
|    min|http://www.amazon...|        AJ Finn|                    |      ACE|  10TH ANNIVERSARY|
|    max|https://www.amazo...|various authors|’Tis for the Rebe...|allantine|               ZOO|
+-------+--------------------+---------------+--------------------+---------+------------------+



In [21]:
# Returns columns of dataframe
dataframe.columns

['_id',
 'amazon_product_url',
 'author',
 'bestsellers_date',
 'description',
 'price',
 'published_date',
 'publisher',
 'rank',
 'rank_last_week',
 'title',
 'weeks_on_list']

In [22]:
# Counts the number of rows in dataframe
dataframe.count()

10195

In [23]:
# Counts the number of distinct rows in dataframe
dataframe.distinct().count()

10195

In [24]:
# Prints plans including physical and logical
dataframe.explain(3)

== Parsed Logical Plan ==
Relation[_id#36,amazon_product_url#37,author#38,bestsellers_date#39,description#40,price#41,published_date#42,publisher#43,rank#44,rank_last_week#45,title#46,weeks_on_list#47] json

== Analyzed Logical Plan ==
_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>
Relation[_id#36,amazon_product_url#37,author#38,bestsellers_date#39,description#40,price#41,published_date#42,publisher#43,rank#44,rank_last_week#45,title#46,weeks_on_list#47] json

== Optimized Logical Plan ==
Relation[_id#36,amazon_product_url#37,author#38,bestsellers_date#39,description#40,price#41,published_date#42,publisher#43,rank#44,rank_la

# ADD, UPDATE & REMOVE COLUMNS

In [26]:
# Lit() is required while we are creating columns with exact values.

dataframe = dataframe.withColumn('new_column', lit('This is a new column'))
dataframe.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

In [27]:
# Update column 'amazon_product_url' with 'URL'

dataframe = dataframe.withColumnRenamed('amazon_product_url', 'URL')
dataframe.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

In [28]:
dataframe_remove = dataframe.drop("publisher", "published_date").show(5)

dataframe_remove2 = dataframe.drop(dataframe.publisher).drop(dataframe.published_date).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]| [2]|           [1]|            THE HOST|          [3]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|     Emily Giffin|[[1211587

# GROUPBY

In [29]:
# Group by author, count the books of the authors in the groups
dataframe.groupBy("rank_last_week").count().show(10)

+--------------+-----+
|rank_last_week|count|
+--------------+-----+
|           [7]|  470|
|          [10]|  363|
|          [15]|  123|
|          [11]|  327|
|           [4]|  524|
|           [3]|  520|
|          [12]|  289|
|           [6]|  497|
|          [13]|  246|
|           [8]|  460|
+--------------+-----+
only showing top 10 rows



# MISIING AND REPLACING VALUES

In [30]:
# Replace null values
dataframe.na.fill(50).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

In [31]:
# Return new dataframe restricting rows with null values
dataframe.na.drop().show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

In [32]:
# Return new dataframe replacing one value with another
dataframe.na.replace(10, 20).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

In [33]:
dataframe.na.replace(['_id', 'Unique_ID'], ['amazon_product_url', 'URL'], 'name').show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

# REPARTITIONING

In [34]:
# Dataframe with 10 partitions

dataframe.repartition(10).rdd.getNumPartitions()

10

In [35]:
# Dataframe with 1 partition

dataframe.coalesce(1).rdd.getNumPartitions()

1

# RUNNING SQL QUERIES PROGRAMMATICALLY

# Registering DataFrames 

In [36]:
dataframe.createGlobalTempView("nyt2")

In [37]:
dataframe.createTempView("bestsellers")

In [38]:
dataframe.createOrReplaceTempView("nyt2")

# Registering tables

In [39]:
# Registering a table
dataframe.registerTempTable("df")

In [40]:
sc.sql("select * from df").show(3)


sc.sql("select \
               CASE WHEN description LIKE '%love%' THEN 'Love_Theme' \
               WHEN description LIKE '%hate%' THEN 'Hate_Theme' \
               WHEN description LIKE '%happy%' THEN 'Happiness_Theme' \
               WHEN description LIKE '%anger%' THEN 'Anger_Theme' \
               WHEN description LIKE '%horror%' THEN 'Horror_Theme' \
               WHEN description LIKE '%death%' THEN 'Criminal_Theme' \
               WHEN description LIKE '%detective%' THEN 'Mystery_Theme' \
               ELSE 'Other_Themes' \
               END Themes \
       from df").groupBy('Themes').count().show()

+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Bro

# OUTPUT

# Data Structures

In [41]:
# Converting dataframe into an RDD
rdd_convert = dataframe.rdd


In [42]:
# Converting dataframe into a RDD of string
dataframe.toJSON().first()


'{"_id":{"$oid":"5b4aa4ead3089013507db18b"},"URL":"http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20","author":"Dean R Koontz","bestsellers_date":{"$date":{"$numberLong":"1211587200000"}},"description":"Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.","price":{"$numberInt":"27"},"published_date":{"$date":{"$numberLong":"1212883200000"}},"publisher":"Bantam","rank":{"$numberInt":"1"},"rank_last_week":{"$numberInt":"0"},"title":"ODD HOURS","weeks_on_list":{"$numberInt":"1"},"new_column":"This is a new column"}'

In [44]:
# Obtaining contents of df as Pandas dataFrame
dataframe.toPandas()


,_id,URL,author,bestsellers_date,description,price,published_date,publisher,rank,rank_last_week,title,weeks_on_list,new_column
0,"(5b4aa4ead3089013507db18b,)",http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"((1211587200000,),)","Odd Thomas, who can communicate with the dead,...","(None, 27)","((1212883200000,),)",Bantam,"(1,)","(0,)",ODD HOURS,"(1,)",This is a new column
1,"(5b4aa4ead3089013507db18c,)",http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,"((1211587200000,),)",Aliens have taken control of the minds and bod...,"(25.99, None)","((1212883200000,),)","Little, Brown","(2,)","(1,)",THE HOST,"(3,)",This is a new column
2,"(5b4aa4ead3089013507db18d,)",http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,"((1211587200000,),)",A woman's happy marriage is shaken when she en...,"(24.95, None)","((1212883200000,),)",St. Martin's,"(3,)","(2,)",LOVE THE ONE YOU'RE WITH,"(2,)",This is a new column
3,"(5b4aa4ead3089013507db18e,)",http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,"((1211587200000,),)",A Massachusetts state investigator and his tea...,"(22.95, None)","((1212883200000,),)",Putnam,"(4,)","(0,)",THE FRONT,"(1,)",This is a new column
4,"(5b4aa4ead3089013507db18f,)",http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,"((1211587200000,),)",An aging porn queens aims to cap her career by...,"(24.95, None)","((1212883200000,),)",Doubleday,"(5,)","(0,)",SNUFF,"(1,)",This is a new column
5,"(5b4aa4ead3089013507db190,)",http://www.amazon.com/Sundays-at-Tiffanys-Jame...,James Patterson and Gabrielle Charbonnet,"((1211587200000,),)",A woman finds an unexpected love,"(24.99, None)","((1212883200000,),)","Little, Brown","(6,)","(3,)",SUNDAYS AT TIFFANY’S,"(4,)",This is a new column
6,"(5b4aa4ead3089013507db191,)",http://www.amazon.com/Phantom-Prey-John-Sandfo...,John Sandford,"((1211587200000,),)",The Minneapolis detective Lucas Davenport inve...,"(26.95, None)","((1212883200000,),)",Putnam,"(7,)","(4,)",PHANTOM PREY,"(3,)",This is a new column
7,"(5b4aa4ead3089013507db192,)",http://www.amazon.com/From-Worse-Southern-Vamp...,Jimmy Buffett,"((1211587200000,),)",A Southern family tries to hide its pet pig at...,"(21.99, None)","((1212883200000,),)","Little, Brown","(8,)","(6,)",SWINE NOT?,"(2,)",This is a new column
8,"(5b4aa4ead3089013507db193,)",http://www.amazon.com/Where-Are-You-Now-Novel/...,Elizabeth George,"((1211587200000,),)","In Cornwall, trying to recover from his wife's...","(27.95, None)","((1212883200000,),)",Harper,"(9,)","(8,)",CARELESS IN RED,"(3,)",This is a new column
9,"(5b4aa4ead3089013507db194,)",http://www.amazon.com/The-Whole-Truth-David-Ba...,David Baldacci,"((1211587200000,),)",An intelligence agent and a journalist team up...,"(26.99, None)","((1212883200000,),)",Grand Central,"(10,)","(7,)",THE WHOLE TRUTH,"(5,)",This is a new column


# Write & Save Files

In [43]:
# Write & Save File in .parquet format
dataframe.select("author", "title", "rank", "description")\
.write \
.save("Rankings_Descriptions.parquet")

In [44]:
# Write & Save File in .json format
dataframe.select("author", "title") \
.write \
.save("Authors_Titles.json",format="json")

# STOPPING SPARK SESSION

In [45]:
# End Spark Session 
sc.stop()